In [ ]:
import json
import pandas as pd
import json
from typing import List, Dict, Optional
from groq import Groq

## GPT using Groq

In [ ]:
import os
from groq import Groq
from openai import OpenAI

import os
import openai

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key= os.getenv("GROQ_API_KEY"),
)

client = Groq(api_key=os.getenv("GROQ_API_KEY"))
client1 = OpenAI(api_key= os.getenv("openai_api_key"))

file_path = "/home/nebulamind/Documents/AI Lab/AskDeen/prepared_outputs/gpt_batch_001.jsonl"
response = client.files.create(file=open(file_path, "rb"), purpose="batch")

print(response)

In [ ]:

OUTPUT_DIR = "./outputs_groq"
os.makedirs(OUTPUT_DIR, exist_ok=True)

job_ids = [
    # "e.g. batch_hg56fgf65fdhf56hdf"
]

def download_outputs():
    all_data = []

    for job_id in job_ids:
        print(f"📥 Fetching job {job_id}")
        batch = client.batches.retrieve(job_id)
        file_id = batch.output_file_id

        if not file_id:
            print(f"⚠️ No output file yet for job {job_id}")
            continue

        content = client1.files.content(file_id).text
        with open(f"{OUTPUT_DIR}/{job_id}_output.jsonl", "w", encoding="utf-8") as f:
            f.write(content)

        for line in content.strip().split("\n"):
            record = json.loads(line)
            custom_id = record["custom_id"]
            translated = record["response"]["body"]["choices"][0]["message"]["content"]
            all_data.append({"custom_id": custom_id, "translation": translated})

    df = pd.DataFrame(all_data)
    df.to_csv(f"{OUTPUT_DIR}/all_translations.csv", index=False)
    print("✅ Results saved to all_translations.csv")

if __name__ == "__main__":
    download_outputs()

## GPT using OpenAI batch API

In [ ]:
from openai import OpenAI

batch_input_file = client1.files.create(
    file=open("/home/nebulamind/Documents/AI Lab/AskDeen/prepared_outputs/gpt_batch_001.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file)

In [ ]:
from openai import OpenAI
# client = OpenAI()

batch_input_file_id = batch_input_file.id
client1.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "nightly eval job"
    }
)

In [ ]:
#chech the batch status

batch = client.batches.retrieve("batch_abc123")
print(batch)

In [ ]:
#get response of batch file 

file_response = client.files.content("file-xyz123")
print(file_response.text)

## ---


In [ ]:
# import os
# import json
# import openai

# # Set your environment variable for GROQ API Key or manually assign it


# # Create OpenAI client with Groq endpoint
# client = openai.OpenAI(
#     base_url="https://api.groq.com/openai/v1",
#     api_key=os.environ.get("GROQ_API_KEY")
# )

# # Constants
# JSONL_FILE_PATH = "/home/nebulamind/Documents/AI Lab/AskDeen/prepared_outputs/gpt_batch_test.jsonl"
# MODEL_NAME = "gpt-3.5-turbo"

# # Load batch data from JSONL
# def load_jsonl(file_path):
#     with open(file_path, "r", encoding="utf-8") as f:
#         return [json.loads(line.strip()) for line in f.readlines()]

# # Perform batch inference
# def run_batch_inference(data, model=MODEL_NAME):
#     results = []

#     for idx, item in enumerate(data):
#         try:
#             response = client.chat.completions.create(
#                 model=model,
#                 messages=item["messages"]
#             )
#             output = response.choices[0].message.content
#             results.append({
#                 "input": item,
#                 "output": output
#             })
#             print(f"✓ Processed item {idx + 1}/{len(data)}")
#         except Exception as e:
#             print(f"✗ Error on item {idx + 1}: {e}")
#             results.append({
#                 "input": item,
#                 "output": None,
#                 "error": str(e)
#             })

#     return results

# # Run the process
# if __name__ == "__main__":
#     batch_data = load_jsonl(JSONL_FILE_PATH)
#     outputs = run_batch_inference(batch_data)

#     # Save results
#     with open("groq_batch_outputs.json", "w", encoding="utf-8") as f:
#         json.dump(outputs, f, indent=2, ensure_ascii=False)

#     print("✅ Batch inference complete. Output saved to groq_batch_outputs.json")


## ---

In [ ]:
import os, json
from groq import Groq
from time import sleep


client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

# client = Groq(api_key=os.environ["GROQ_API_KEY"])
input_path = "/home/nebulamind/Documents/AI Lab/AskDeen/prepared_outputs/gpt_batch_test.jsonl"
output_path = "/home/nebulamind/Documents/AI Lab/AskDeen/prepared_outputs/llama3_responses.jsonl"

with open(input_path, "r", encoding="utf-8") as fin, open(output_path, "w", encoding="utf-8") as fout:
    for line in fin:
        record = json.loads(line)
        msgs = record["messages"]
        resp = client.chat.completions.create(
            model="meta-llama/llama-4-maverick-17b-128e-instruct",
            messages=msgs
        )
        output = {
            "input": record,
            "output": resp.choices[0].message.content
        }
        fout.write(json.dumps(output, ensure_ascii=False) + "\n")
        sleep(0.3)  # throttle as needed
